In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('CUDA version:', torch.version.cuda)
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce GTX 960
CUDA version: 11.7
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
import numpy as np
import PIL
from PIL import Image
import datasets
import os
import evaluate
import torch
import json
import codecs

c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
seed = 9876

image_folder = './WE3DS/images/'
annotation_folder = './WE3DS/annotations/'
annotations_aggregated_folder = './WE3DS/annotations_aggregated/'
# Define the paths to the images and annotations
all_image_names = np.array(os.listdir(image_folder))

In [6]:
plant_classification = {
    'void': 'void',
    'soil': 'soil',
    'broad bean': 'crop',
    'corn spurry': 'weed',
    'red-root amaranth': 'weed',
    'common buckwheat': 'crop',
    'pea': 'crop',
    'red fingergrass': 'weed',
    'common wild oat': 'weed',
    'cornflower': 'weed',
    'corn cockle': 'weed',
    'corn': 'crop',
    'milk thistle': 'weed',
    'rye brome': 'weed',
    'soybean': 'crop',
    'sunflower': 'crop',
    'narrow-leaved plantain': 'weed',
    'small-flower geranium': 'weed',
    'sugar beet': 'crop'
}

crop_indices = [index for index, value in enumerate(plant_classification) if plant_classification[value] == 'crop']
weed_indices = [index for index, value in enumerate(plant_classification) if plant_classification[value] == 'weed']

print("Crop indicies: ", crop_indices)
print("Weed indicies: ", weed_indices)

Crop indicies:  [2, 5, 6, 11, 14, 15, 18]
Weed indicies:  [3, 4, 7, 8, 9, 10, 12, 13, 16, 17]


In [7]:
# This is a configuration that will determine which type of model and for which plants the following code will be executed
weed_plants = [plant_name.replace(" ", "_") for plant_name, classification in plant_classification.items() if classification == 'weed']
# weed_plants = []

crop = 'broad_bean'
# crop = 'common_buckwheat'
# crop = 'pea'
# crop = 'corn'
# crop = 'soybean'
# crop = 'sunflower'
# crop = 'sugar_beet'

# model_type = 'multiclass'
model_type = 'binary'

model_plant_names = [crop] + weed_plants
print(model_plant_names)

['broad_bean', 'corn_spurry', 'red-root_amaranth', 'red_fingergrass', 'common_wild_oat', 'cornflower', 'corn_cockle', 'milk_thistle', 'rye_brome', 'narrow-leaved_plantain', 'small-flower_geranium']


In [8]:
def get_image_meta_filepath(plant_name):
    suffix = '_images.json'
    if plant_name in weed_plants:
        suffix = '_no_crop_images.json'
    return './meta/' + plant_name + suffix

In [9]:
def get_image_list_for_plant(plant_name, model_type):
    # Create an empty list to store the dataset
    image_list = []
    plant_image_names = json.load(codecs.open(get_image_meta_filepath(plant_name), 'r', 'utf-8-sig'))

    # Exclude images that contain more than one crop
    image_names_to_exclude = ['img_01096.png' 'img_01098.png']
    plant_image_names = [image_name for image_name in plant_image_names if image_name not in image_names_to_exclude]
    print(plant_image_names)

    # Iterate over the image and annotation paths
    for image_name in plant_image_names:
        # Load the image and annotation using PIL
        image = Image.open(image_folder + image_name)
        path = None
        if model_type == 'multiclass':
            path = 'WE3DS/annotations_multiclass/' + crop + '/' + image_name
        elif model_type == 'binary':
            path = 'WE3DS/annotations_binary/' + crop + '/' + image_name

        annotation = Image.open(path)
        
        # Create a dictionary entry for the dataseta
        entry = {'image': image, 'annotation': annotation}
        
        # Add the entry to the dataset
        image_list.append(entry)

    return image_list

In [10]:
def create_and_split_dataset_for_plant(plant_image_list):
    dataset = datasets.Dataset.from_list(plant_image_list)
    dataset = dataset.train_test_split(test_size=0.5, seed=seed)
    train_ds = dataset["train"]
    val_ds, test_ds = dataset["test"].train_test_split(test_size=0.5, seed=seed).values()
    return train_ds, val_ds, test_ds

In [11]:
def create_datasets_for_plants(plant_names, model_type):
    p0_image_list = get_image_list_for_plant(plant_names[0], model_type)
    print("Number of plant images for plant", plant_names[0], ":", len(p0_image_list))
    train_ds, val_ds, test_ds = create_and_split_dataset_for_plant(p0_image_list)

    for plant_name in plant_names[1:]:
        p_image_list = get_image_list_for_plant(plant_name, model_type)
        print("Number of plant images for plant", plant_name, ":", len(p_image_list))
        p_train_ds, p_val_ds, p_test_ds = create_and_split_dataset_for_plant(p_image_list)

        train_ds = datasets.concatenate_datasets([train_ds, p_train_ds])
        val_ds = datasets.concatenate_datasets([val_ds, p_val_ds])
        test_ds = datasets.concatenate_datasets([test_ds, p_test_ds])

    return train_ds, val_ds, test_ds

In [12]:
train_ds, val_ds, test_ds = create_datasets_for_plants(model_plant_names, model_type)

['img_00173.png', 'img_00174.png', 'img_00175.png', 'img_00176.png', 'img_00177.png', 'img_00178.png', 'img_00672.png', 'img_00673.png', 'img_00674.png', 'img_00675.png', 'img_00676.png', 'img_00677.png', 'img_00678.png', 'img_00679.png', 'img_00680.png', 'img_00681.png', 'img_00682.png', 'img_00683.png', 'img_00684.png', 'img_00882.png', 'img_00883.png', 'img_00884.png', 'img_00885.png', 'img_00886.png', 'img_00887.png', 'img_00938.png', 'img_00980.png', 'img_00981.png', 'img_00982.png', 'img_00983.png', 'img_00984.png', 'img_00985.png', 'img_00986.png', 'img_00987.png', 'img_00988.png', 'img_00989.png', 'img_01070.png', 'img_01071.png', 'img_01072.png', 'img_01073.png', 'img_01074.png', 'img_01075.png', 'img_01076.png', 'img_01077.png', 'img_01078.png', 'img_01079.png', 'img_01219.png', 'img_01220.png', 'img_01221.png', 'img_01222.png', 'img_01223.png', 'img_01224.png', 'img_01225.png', 'img_01226.png', 'img_01227.png', 'img_01228.png', 'img_01279.png', 'img_01280.png', 'img_01281.pn

In [13]:
print("Training subset number of images: " + str(train_ds.num_rows))
print("Validation subset number of images: " + str(val_ds.num_rows))
print("Test subset number of images: " + str(test_ds.num_rows))

Training subset number of images: 467
Validation subset number of images: 232
Test subset number of images: 237


In [14]:
from transformers import AutoImageProcessor
from transformers import SegformerImageProcessor

checkpoint = "nvidia/mit-b0"
image_processor = SegformerImageProcessor.from_pretrained(checkpoint)
image_processor

c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\transformers\models\segformer\image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


SegformerImageProcessor {
  "do_normalize": true,
  "do_reduce_labels": false,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "SegformerFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "SegformerImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 512,
    "width": 512
  }
}

In [15]:
def train_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels, return_tensors="pt")
    inputs
    return inputs

In [16]:
metric = evaluate.load("mean_iou")

In [17]:
labels = ['void', 'soil', crop]

if model_type == 'binary':
    labels.append('weeds')
elif model_type == 'multiclass':
    for weed_plant in weed_plants:
        labels.append(weed_plant)

ids = list(range(0, len(labels)))

id2label = dict(zip(ids, labels))
label2id = dict(zip(labels, ids))

num_labels = len(labels)

print('labels:', labels)
print('ids:', ids)
print('num_labels:', num_labels)
print('id2label:', id2label)
print('label2id:', label2id)

labels: ['void', 'soil', 'broad_bean', 'weeds']
ids: [0, 1, 2, 3]
num_labels: 4
id2label: {0: 'void', 1: 'soil', 2: 'broad_bean', 3: 'weeds'}
label2id: {'void': 0, 'soil': 1, 'broad_bean': 2, 'weeds': 3}


In [18]:
def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = torch.nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [19]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer, EarlyStoppingCallback

model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)
model.to(device)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.running_mean', 'decode_head.linear_c.2.proj.weight', 'decode_head.classifier.weight', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.3.proj.bias', 'decode_head.batch_norm.weight', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.1.proj.weight', 'decode_head.batch_norm.bias', 'decode_head.linear_c.1.proj.bias', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.3.proj.weight', 'decode_head.classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SegformerForSemanticSegmentation(
  (segformer): SegformerModel(
    (encoder): SegformerEncoder(
      (patch_embeddings): ModuleList(
        (0): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(3, 32, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (layer_norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        )
        (1): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (2): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(64, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((160,), eps=1e-05, elementwise_affine=True)
        )
        (3): SegformerOverlapPatchEmbeddings(
          (proj): Conv2d(160, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  

In [20]:
# TODO try to remove eval_accumulation_steps
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=25,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    eval_steps=100,
    logging_steps=1,
    # eval_accumulation_steps=5,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    seed=seed,
)

In [21]:
def initialize_trainer(train_ds, test_ds) :
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    return trainer

In [22]:
train_ds.set_transform(train_transforms)
val_ds.set_transform(train_transforms)
test_ds.set_transform(train_transforms)

In [23]:
trainer = initialize_trainer(train_ds, val_ds)
trainer.train()

c:\Users\Kate\AppData\Local\miniconda3\envs\master\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 1/1175 [00:09<3:11:17,  9.78s/it]

{'loss': 1.3335, 'learning_rate': 5.9948936170212765e-05, 'epoch': 0.02}


  0%|          | 2/1175 [00:17<2:43:16,  8.35s/it]

{'loss': 1.2963, 'learning_rate': 5.9897872340425535e-05, 'epoch': 0.04}


  0%|          | 3/1175 [00:24<2:33:03,  7.84s/it]

...

100%|█████████▉| 1174/1175 [3:21:22<00:07,  7.11s/it]

{'loss': 0.0298, 'learning_rate': 5.1063829787234044e-08, 'epoch': 24.98}


100%|██████████| 1175/1175 [3:21:26<00:00, 10.29s/it]

{'loss': 0.0336, 'learning_rate': 0.0, 'epoch': 25.0}
{'train_runtime': 12086.4048, 'train_samples_per_second': 0.966, 'train_steps_per_second': 0.097, 'train_loss': 0.19395920311199857, 'epoch': 25.0}


TrainOutput(global_step=1175, training_loss=0.19395920311199857, metrics={'train_runtime': 12086.4048, 'train_samples_per_second': 0.966, 'train_steps_per_second': 0.097, 'train_loss': 0.19395920311199857, 'epoch': 25.0})

In [24]:
trainer.state.log_history

[{'loss': 1.3335,
  'learning_rate': 5.9948936170212765e-05,
  'epoch': 0.02,
  'step': 1},
 {'loss': 1.2963,
  'learning_rate': 5.9897872340425535e-05,
  'epoch': 0.04,
  'step': 2},
 {'loss': 1.2806,
  'learning_rate': 5.98468085106383e-05,
  'epoch': 0.06,
  'step': 3},
 {'loss': 1.2593,
  'learning_rate': 5.979574468085107e-05,
  'epoch': 0.09,
  'step': 4},
 {'loss': 1.2451,
  'learning_rate': 5.974468085106383e-05,
  'epoch': 0.11,
  'step': 5},
 {'loss': 1.2246,
  'learning_rate': 5.9693617021276594e-05,
  'epoch': 0.13,
  'step': 6},
 {'loss': 1.224,
  'learning_rate': 5.9642553191489364e-05,
  'epoch': 0.15,
  'step': 7},
 {'loss': 1.2089,
  'learning_rate': 5.959148936170213e-05,
  'epoch': 0.17,
  'step': 8},
 {'loss': 1.1796,
  'learning_rate': 5.95404255319149e-05,
  'epoch': 0.19,
  'step': 9},
 {'loss': 1.1972,
  'learning_rate': 5.948936170212766e-05,
  'epoch': 0.21,
  'step': 10},
 {'loss': 1.1453,
  'learning_rate': 5.943829787234043e-05,
  'epoch': 0.23,
  'step': 1

In [25]:
# Save the trained model, so that it can be used for inference later.
# Save the log history, so that it can be used for plotting later.
trainer.save_model('models/' + model_type + '/' + crop)
with open('models/' + model_type + '/' + crop + '/log_history.json', 'w') as file:
    log_history = trainer.state.log_history
    json.dump(log_history, file)

In [26]:
outputs = trainer.predict(test_ds)

100%|██████████| 24/24 [05:42<00:00, 14.29s/it]


In [27]:
test_metric = trainer.evaluate(test_ds)
test_metric

100%|██████████| 24/24 [05:47<00:00, 14.47s/it]


{'eval_loss': 0.020868008956313133,
 'eval_mean_iou': 0.6542348796868243,
 'eval_mean_accuracy': 0.6993652718220992,
 'eval_overall_accuracy': 0.9957957046444406,
 'eval_per_category_iou': [0.0,
  0.9958969752494089,
  0.8880824785624387,
  0.7329600649354491],
 'eval_per_category_accuracy': [0.0,
  0.9978827335276511,
  0.9456774431485152,
  0.8539009106122303],
 'eval_runtime': 351.294,
 'eval_samples_per_second': 0.675,
 'eval_steps_per_second': 0.068,
 'epoch': 25.0}

In [28]:
with open('models/' + model_type + '/' + crop + '/test_metric.json', 'w') as file:
    json.dump(test_metric, file)

In [29]:
# upsampled_ouputs = torch.nn.functional.interpolate(
#     outputs,
#     size=image.size[::-1],
#     mode="bilinear",
#     align_corners=False,
# )
# y_pred = upsampled_ouputs.predictions.argmax(1)

In [30]:
# print("Predicting following unique classes: ", np.unique(y_pred))
# print(y_pred.shape)

In [31]:
# y_pred

In [32]:
# trainer.optimizer

In [33]:
# import subprocess
# from typing import NoReturn

# def shutdown_windows() -> NoReturn:
#     subprocess.run(["shutdown", "/s", "/t", "0"])

# shutdown_windows()